In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\OptimizerComparison"
dataframes = {}
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders

['COBYLA',
 'DifferentialEvolutionTuned',
 'DifferentialEvolutionTuned-NoPolish',
 'DifferentialEvolutionUntuned',
 'L-BFGS-B']

In [8]:
converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots_list = []
maxiter = []

shots = '1024'

for folder in folders:
    d_path = os.path.join(folder_path, folder, 'DW', shots, 'DW_16.json')
    
    with open(d_path, 'r') as json_file:
        data = json.load(json_file)

    num_vqe.append(data['num_VQE'])
    shots_list.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['parallel_run_time'])

In [9]:
df = pd.DataFrame({
    'folder': folders,
    'num_vqe': num_vqe,
    'shots': shots_list,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [7]:
df

,folder,num_vqe,shots,Converged Runs,maxiter,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
0,COBYLA,40,None,18/100,10000,7719,8.93326e-01,1.72709e-03,9.13410e-01,2.18102e-02,8.91599e-01,0:00:36.124917
1,DifferentialEvolutionTuned,40,None,40/100,10000,144,8.93291e-01,1.69133e-03,8.93703e-01,2.10356e-03,8.91599e-01,0:00:22.051830
2,DifferentialEvolutionTuned-NoPolish,40,None,40/100,10000,136,8.94056e-01,2.45660e-03,8.95985e-01,4.38543e-03,8.91599e-01,0:00:18.671494
3,DifferentialEvolutionUntuned,40,None,40/100,10000,106,8.93291e-01,1.69133e-03,8.93698e-01,2.09896e-03,8.91599e-01,0:01:15.727444
4,L-BFGS-B,40,None,40/100,10000,430,8.93291e-01,1.69151e-03,8.95034e-01,3.43464e-03,8.91599e-01,0:00:07.916913


In [10]:
df

,folder,num_vqe,shots,Converged Runs,maxiter,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,Time
0,COBYLA,40,1024,40/100,10000,140,1.31232e+00,4.20720e-01,6.65801e+00,5.76641e+00,8.91599e-01,0:00:05.714027
1,DifferentialEvolutionTuned,40,1024,40/100,10000,215,8.91599e-01,2.25375e-14,8.91641e-01,4.13804e-05,8.91599e-01,0:00:39.671451
2,DifferentialEvolutionTuned-NoPolish,40,1024,40/100,10000,259,8.91599e-01,2.25375e-14,8.91641e-01,4.14210e-05,8.91599e-01,0:01:14.693939
3,DifferentialEvolutionUntuned,40,1024,40/100,10000,154,8.91599e-01,2.25375e-14,8.91641e-01,4.12992e-05,8.91599e-01,0:02:18.883809
4,L-BFGS-B,40,1024,32/100,10000,277,1.50460e+01,1.41544e+01,5.52466e+01,5.43550e+01,8.91599e-01,0:00:07.281348
